In [1]:
#Import packages
import geemap
import geopandas
import ee
import os

In [35]:
Map = geemap.Map(center=[20,85], zoom=4)
Map

Map(center=[20, 85], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [44]:
#Load in aoi
bounding_box = 'C:/Users/Jeremy/OneDrive/GIS/Blender/Ananya Map/Output/bounding_box.shp'
aoi = geemap.shp_to_ee(bounding_box)
aoi_geometry = aoi.geometry()

#Add aoi_geometry bounding box to the map
Map.addLayer(aoi_geometry, {}, 'aoi')

#Centre the map over the aoi
Map.centerObject(aoi_geometry, 11) 
  
#Dates over which to create a median composite. Full dates are biased towards summer due to cloud coverage.
start_date = ee.Date('2017-01-01')
end_date = ee.Date('2023-05-30')

#Set cloud filtering variables
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

In [40]:
#Set image collections from GEE
#Sentinel-2 Level 1C data.  Bands B7, B8, B8A and B10 from this dataset are needed as input to CDI and the cloud mask function.
s2 = ee.ImageCollection('COPERNICUS/S2')
#Cloud probability dataset.  The probability band is used in the cloud mask function.
s2c = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
#Sentinel-2 surface reflectance data for the composite.
s2Sr = ee.ImageCollection('COPERNICUS/S2_SR')

In [53]:
#Filter dataset from GEE
#S2 L1C for Cloud Displacement Index (CDI) bands.
s2 = s2.filterBounds(aoi_geometry) \
    .filterDate(start_date, end_date) \
    .select(['B7', 'B8', 'B8A', 'B10'])
#S2Cloudless for the cloud probability band.
s2c = s2c.filterDate(start_date, end_date) \
    .filterBounds(aoi_geometry)
#S2 L2A for surface reflectance bands.
s2Sr = s2Sr.filterDate(start_date, end_date) \
    .filterBounds(aoi_geometry) \
    .select(['B2', 'B3', 'B4', 'B5', 'B6'])

In [59]:
Map.addLayer(s2Sr, {'bands':['B4',  'B3',  'B2'], 'min':0, 'max':3000, 'gama':1.1}, 'Sentinel 2 Composite')

In [60]:
#Join two collections on their 'system:index' property. The propertyName parameter is the name of the property that references the joined image.
def indexJoin(collectionA, collectionB, propertyName):
    # Join the collections on 'system:index'
    joined = ee.ImageCollection(
        ee.Join.saveFirst(propertyName).apply(
            primary=collectionA,
            secondary=collectionB,
            condition=ee.Filter.equals(
                leftField='system:index',
                rightField='system:index'
            )
        )
    )

    # Merge the bands of the joined image
    def mergeBands(image):
        return image.addBands(ee.Image(image.get(propertyName)))
    return joined.map(mergeBands)

In [61]:
def maskImage(image):
    # Compute the cloud displacement index from the L1C bands.
    cdi = ee.Algorithms.Sentinel2.CDI(image)
    s2c = image.select('probability')
    cirrus = image.select('B10').multiply(0.0001)

    # Assume low-to-mid atmospheric clouds to be pixels where probability
    # is greater than 70%, and CDI is less than -0.2. For higher atmosphere
    # cirrus clouds, assume the cirrus band is greater than 0.02.
    # The final cloud mask is one or both of these conditions.
    # The higher the threshold, the more imagery is included and there is less chance of pixels being left na.
    # There is however more chance of cloud being included.
    isCloud = s2c.gt(70).And(cdi.lt(-0.2)).Or(cirrus.gt(0.02))

    # Reproject is required to perform spatial operations at 20m scale.
    # 20m scale is for speed, and assumes clouds don't require 10m precision.
    isCloud = isCloud.focal_min(3).focal_max(16)
    isCloud = isCloud.reproject(crs=cdi.projection(), scale=20)

    # Project shadows from clouds we found in the last step. This assumes we're working in
    # a UTM projection.
    shadowAzimuth = ee.Number(90).subtract(ee.Number(image.get('MEAN_SOLAR_AZIMUTH_ANGLE')))

    # With the following reproject, the shadows are projected 5km.
    isCloud = isCloud.directionalDistanceTransform(shadowAzimuth, 50)
    isCloud = isCloud.reproject(crs=cdi.projection(), scale=100)

    isCloud = isCloud.select('distance').mask()
    return image.select(['B2', 'B3', 'B4', 'B5', 'B6']).updateMask(isCloud.Not())

In [62]:
# Join the cloud probability dataset to surface reflectance.
withCloudProbability = indexJoin(s2Sr, s2c, 'cloud_probability')

# Join the L1C data to get the bands needed for CDI.
withS2L1C = indexJoin(withCloudProbability, s2, 'l1c')

# Map the cloud masking function over the joined collection.
masked = withS2L1C.map(maskImage)

# Take the median, specifying a tileScale to avoid memory errors.
median = masked.median() \
    .clip(aoi_geometry)

In [63]:
Map.addLayer(median, {'bands':['B4',  'B3',  'B2'], 'min':0, 'max':3000, 'gama':1.1}, 'Sentinel 2 Composite')

Below is testing from the Colab script

In [64]:
def get_s2_sr_cld_col(aoi_geometry, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi_geometry)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi_geometry)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [65]:
s2_sr_cld_col = get_s2_sr_cld_col(aoi_geometry, start_date, end_date)

In [66]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [67]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')))

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [68]:
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

In [69]:
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

In [70]:
s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())

In [71]:
Map.addLayer(s2_sr_median, {'bands':['B4',  'B3',  'B2'], 'min':0, 'max':3000, 'gama':1.1}, 'Sentinel 2 Composite')
